In [1]:
import numpy as np
import pickle

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial.distance import cdist, pdist
from sklearn.metrics import accuracy_score

In [2]:
def bipolarize(Y):
    X = np.copy(Y)
    X[X > 0] = 1.0
    X[X < 0] = -1.0
    X[X == 0] = np.random.choice([-1.0, 1.0], size=len(X[X == 0]))
    return X

In [3]:
def centroids(X,label=None,doMean=False):
    if label is not None:
        c = np.zeros((len(np.unique(label)), X.shape[1]))
        for i,l in enumerate(np.unique(label)):
            if doMean:
                c[i,:] = np.mean(X[label==l],axis=0)
            else:
                c[i,:] = bipolarize(np.sum(X[label==l],axis=0))
    else:
        if doMean:
            c = np.mean(X,axis=0).reshape(1,-1)
        else:
            c = bipolarize(np.sum(X,axis=0)).reshape(1,-1)
    return c

In [4]:
# select dataset and encoding type
dataName = 'allHV.npz'
emgHVType =  'hvRel'

allHV = np.load(dataName)

# extract data and labels based on gesture, trial, and position
hv = allHV[emgHVType]
gestLabel = allHV['gestLabel']
posLabel = allHV['posLabel']
trialLabel = allHV['trialLabel']

# get list of unique values for each label
gestures = np.unique(gestLabel)
positions = np.unique(posLabel)
trials = np.unique(trialLabel)

numGestures = len(gestures)
numPositions = len(positions)
numTrials = len(trials)

# get data size info
D = hv.shape[1] # hypervector dimension
numHV = 80 # number of examples per trial

# color palettes for plotting
gPalette = sns.color_palette('Paired', numGestures)
pPalette = sns.color_palette('Paired', numPositions)

In [5]:
allLabels = []
for g in gestures:
    with open('./clusters/labels_' + str(g) + '_test.pickle', 'rb') as f:
        allLabels.append(pickle.load(f))

In [6]:
X = np.copy(hv)

In [7]:
# # numClust = [1,1,1,1,1,1,1,1,1,1,1,1,1]
# numClust = 30*np.ones(numGestures,dtype='int')
# AM = []
# AMLabel = []
# for g in gestures:
#     cIdx = allLabels[g][numClust[g]]
#     c = centroids(X[gestLabel == g], label=cIdx, doMean=False)
#     AM.append(c)
#     for n in range(numClust[g]):
#         AMLabel.append(g)
# AM = np.vstack(AM)
# AMLabel = np.array(AMLabel)
# dist = cdist(X,AM,'hamming')
# pred = AMLabel[np.argmin(dist,axis=1)]
# acc = accuracy_score(pred,gestLabel)

In [8]:
%%time

maxClust = 30
res = []
for g in gestures[9:]:
    Xgest = X[gestLabel == g]
    print('Running gesture %d' % (g))

    distOther = np.zeros((maxClust,len(Xgest)))

    # get the prototypes for all other labels with varying number of clusters
    for numClust in range(1,maxClust + 1):
        AMOther = []
        for gOther in list(set(gestures) - set([g])):
    #         print('Getting centroids for gesture %d with %d clusters... ' % (gOther,numClust), end="")
            cIdx = allLabels[gOther][numClust]
            AMOther.append(centroids(X[gestLabel==gOther], label=cIdx, doMean=False))
    #         print('Done!')
        AMOther = np.vstack(AMOther)
        print('Getting minimum distances from gesture %d examples with %d centroids for each other gesture... ' % (g,numClust), end="")
        distOther[numClust-1,:] = np.min(cdist(Xgest,AMOther,'hamming'),axis=1)
        print('Done!')

    # now loop through number of clusters to use for the tested gesture
    acc = np.zeros((maxClust,maxClust))
    distSelf = np.zeros((maxClust,len(Xgest)))
    for numClust in range(1,maxClust + 1):
        print('Getting leave-one-out centroid classification for %d clusters... ' % (numClust))
        cIdx = allLabels[g][numClust]
        for i,xTest in enumerate(Xgest):
            AM = centroids(Xgest[np.arange(len(Xgest)) != i], label=cIdx[np.arange(len(Xgest)) != i], doMean=False)
            distSelf[numClust-1,i] = np.min(cdist(xTest.reshape(1,-1),AM,'hamming'))
        print(['{:.3f}'.format(len(np.where(distSelf[numClust-1] < distOther[x])[0])/len(Xgest)) for x in range(maxClust)])
        acc[numClust-1,:] = np.array([len(np.where(distSelf[numClust-1] < distOther[x])[0])/len(Xgest) for x in range(maxClust)])
        
    res.append({'distOther':distOther, 'distSelf':distSelf, 'acc':acc})

Running gesture 9
Getting minimum distances from gesture 9 examples with 1 centroids for each other gesture... Done!
Getting minimum distances from gesture 9 examples with 2 centroids for each other gesture... Done!
Getting minimum distances from gesture 9 examples with 3 centroids for each other gesture... Done!
Getting minimum distances from gesture 9 examples with 4 centroids for each other gesture... Done!
Getting minimum distances from gesture 9 examples with 5 centroids for each other gesture... Done!
Getting minimum distances from gesture 9 examples with 6 centroids for each other gesture... Done!
Getting minimum distances from gesture 9 examples with 7 centroids for each other gesture... Done!
Getting minimum distances from gesture 9 examples with 8 centroids for each other gesture... Done!
Getting minimum distances from gesture 9 examples with 9 centroids for each other gesture... Done!
Getting minimum distances from gesture 9 examples with 10 centroids for each other gesture.

In [9]:
with open('./run9-12.pickle', 'wb') as f: 
    pickle.dump(res,f,pickle.HIGHEST_PROTOCOL)